# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [14]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [15]:
# Store filepath in a variable
weather_data = "../output_data/weather_data.csv"
# Read our Data file with the pandas library
# Not every CSV requires an encoding, but be aware this can come up
weather_data_df = pd.read_csv(weather_data)
# Show just the header

weather_data_df = weather_data_df.drop('Unnamed: 0', 1)
weather_data_df.head()

,City,Clouds,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Avarua,100,CK,1591809884,73,-21.21,-159.78,77.00,11.41
1,Deputatsky,91,RU,1591809884,82,69.30,139.90,36.43,6.64
2,Mahébourg,75,MU,1591809884,78,-20.41,57.70,71.60,18.34
3,Saint-Philippe,20,RE,1591809884,73,-21.36,55.77,71.60,19.46
4,Ambilobe,0,MG,1591809884,74,-13.20,49.05,70.48,3.80


In [16]:
#coming back here because i'm told humidity isn't doing well in the gmap fig
dataTypeSeries = weather_data_df.dtypes
print('Data type of each column of Dataframe :')
print(dataTypeSeries)

Data type of each column of Dataframe :
City           object
Clouds          int64
Country        object
Date            int64
Humidity        int64
Lat           float64
Lng           float64
Max Temp      float64
Wind Speed    float64
dtype: object


In [17]:
weather_data_df['Humidity'] = weather_data_df['Humidity'].astype(float)
#did it work to change to float?
dataTypeSeries = weather_data_df.dtypes
print('Data type of each column of Dataframe :')
print(dataTypeSeries)

Data type of each column of Dataframe :
City           object
Clouds          int64
Country        object
Date            int64
Humidity      float64
Lat           float64
Lng           float64
Max Temp      float64
Wind Speed    float64
dtype: object


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [18]:
gmaps.configure(api_key=g_key)

locations = weather_data_df[['Lat','Lng']]
weights = weather_data_df["Humidity"]
fig = gmaps.figure(map_type="SATELLITE")
fig.add_layer(gmaps.heatmap_layer(locations,weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [19]:
# Narrow down the DataFrame to find your ideal weather condition.
filtered_weather_df = weather_data_df
# Drop any rows that don't contain all three conditions. Want to be sure the weather is ideal.
# A max temperature lower than 80 degrees but higher than 70.
filtered_weather_df = filtered_weather_df.loc[(filtered_weather_df["Max Temp"] < 80) & (filtered_weather_df["Max Temp"] > 70)]
# Wind speed less than 10 mph.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Wind Speed"] < 10]
# Zero clouds
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Clouds"] == 0]
# Drop any rows with null values
filtered_weather_df = filtered_weather_df.dropna()

filtered_weather_df

,City,Clouds,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
4,Ambilobe,0,MG,1591809884,74.0,-13.20,49.05,70.48,3.80
8,Bektemir,0,UZ,1591809884,31.0,41.21,69.33,78.80,9.17
92,Malanje,0,AO,1591809888,37.0,-9.54,16.34,70.66,2.10
250,Kiryat Gat,0,IL,1591809896,65.0,31.61,34.76,77.00,6.42
262,Namie,0,JP,1591809896,84.0,37.48,141.00,72.00,6.85
305,Farah,0,AF,1591809898,18.0,32.50,62.50,75.04,8.34
349,Ruska Poliana,0,UA,1591809901,73.0,49.42,31.92,75.67,7.16
424,Gornopravdinsk,0,RU,1591809905,51.0,60.05,69.90,70.50,6.58
502,Šibenik,0,HR,1591809909,56.0,43.73,15.91,71.60,6.93


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [20]:
hotel_df = filtered_weather_df
hotel_df["Hotel Name"]=""
hotel_df.head()

,City,Clouds,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
4,Ambilobe,0,MG,1591809884,74.0,-13.20,49.05,70.48,3.80,
8,Bektemir,0,UZ,1591809884,31.0,41.21,69.33,78.80,9.17,
92,Malanje,0,AO,1591809888,37.0,-9.54,16.34,70.66,2.10,
250,Kiryat Gat,0,IL,1591809896,65.0,31.61,34.76,77.00,6.42,
262,Namie,0,JP,1591809896,84.0,37.48,141.00,72.00,6.85,


In [26]:
#Plot the hotels on top of the humidity heatmap
#each pin containing the **Hotel Name**, **City**, and **Country**.

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")




Retrieving Results for Index 4: Ambilobe.
Closest hotel is Hôtel & Restaurant DIANA Ambilobe.
------------
Retrieving Results for Index 8: Bektemir.
Closest hotel is Hayot Hotel.
------------
Retrieving Results for Index 92: Malanje.
Closest hotel is Malanje Hotel.
------------
Retrieving Results for Index 250: Kiryat Gat.
Closest hotel is רגע בנחת.
------------
Retrieving Results for Index 262: Namie.
Closest hotel is いこいの村なみえ(福島県).
------------
Retrieving Results for Index 305: Farah.
Missing field/result... skipping.
------------
Retrieving Results for Index 349: Ruska Poliana.
Missing field/result... skipping.
------------
Retrieving Results for Index 424: Gornopravdinsk.
Missing field/result... skipping.
------------
Retrieving Results for Index 502: Šibenik.
Closest hotel is Hotel Jadran.
------------


In [27]:
hotel_df

,City,Clouds,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
4,Ambilobe,0,MG,1591809884,74.0,-13.20,49.05,70.48,3.80,Hôtel & Restaurant DIANA Ambilobe
8,Bektemir,0,UZ,1591809884,31.0,41.21,69.33,78.80,9.17,Hayot Hotel
92,Malanje,0,AO,1591809888,37.0,-9.54,16.34,70.66,2.10,Malanje Hotel
250,Kiryat Gat,0,IL,1591809896,65.0,31.61,34.76,77.00,6.42,רגע בנחת
262,Namie,0,JP,1591809896,84.0,37.48,141.00,72.00,6.85,いこいの村なみえ(福島県)
305,Farah,0,AF,1591809898,18.0,32.50,62.50,75.04,8.34,
349,Ruska Poliana,0,UA,1591809901,73.0,49.42,31.92,75.67,7.16,
424,Gornopravdinsk,0,RU,1591809905,51.0,60.05,69.90,70.50,6.58,
502,Šibenik,0,HR,1591809909,56.0,43.73,15.91,71.60,6.93,Hotel Jadran


In [30]:
# Create locations from the hotel_df to create the markers.
locations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations)


# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
#create the hotels
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# Add marker layer ontop of heat map


# Display figure
